In [1]:
import pandas as pd
import numpy as np


#visualization
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBaclend.figure_format = 'svg'


#preprocessing tools

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer #equal to get dummies
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder


In [2]:
diamonds = pd.read_csv('./data/diamonds_train.csv.zip')
diamonds_predict = pd.read_csv('./data/diamonds_predict.csv')

In [13]:
TARGET = 'price' 


NUM_FEATS = ['carat', 'depth', 'x', 'y', 'z']
CAT_FEATS = ['cut', 'color', 'clarity']
FEATS = NUM_FEATS + CAT_FEATS

In [14]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
               ('scaler', StandardScaler())])

In [15]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
               ('onehot', OneHotEncoder(handle_unknown='ignore'))])#handle unkown dont compute unkown values

In [16]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                               ('cat', categorical_transformer, CAT_FEATS)])

In [17]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'x', 'y', 'z']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['cut', 'color', 'clarity'])])

In [18]:
pd.DataFrame(data=preprocessor.fit_transform(diamonds)).head()

#.todense(), to be able to see the dataframe properly and not tabular manner

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.867006,0.452019,0.978807,0.921985,1.022657,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.004557,0.871099,-1.226738,-1.179816,-1.129259,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.184434,2.617265,-0.097286,-0.176882,0.161891,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.815298,1.429872,-0.933258,-0.883296,-0.770607,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.467458,-0.875068,0.729794,0.677793,0.592274,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.model_selection import train_test_split

In [20]:
diamonds_train, diamonds_test = train_test_split(diamonds)

In [21]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 10)
(10114, 10)


In [22]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

model = Pipeline(steps=[('preprocessor',preprocessor),
                       ('regressor', RandomForestRegressor())])

In [23]:
model.fit(diamonds_train[FEATS], diamonds_train[TARGET]);

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [25]:
y_test = model.predict(diamonds_test[FEATS])
y_train = model.predict(diamonds_train[FEATS])


In [26]:
print(f"Test error: {mean_squared_error(y_pred=y_test,y_true=diamonds_test[TARGET], squared=False)}")
print(f"Train error: {mean_squared_error(y_pred=y_train,y_true=diamonds_train[TARGET], squared=False)}")

Test error: 551.5076144225151
Train error: 213.43525304542143


In [28]:
print(f"test mean sq error: {r2_score(y_pred=y_test, y_true=diamonds_test[TARGET])}")
print(f"train mean sq error: {r2_score(y_pred=y_train, y_true=diamonds_train[TARGET])}")

test mean sq error: 0.9806692867977171
train mean sq error: 0.9971540499096923


In [29]:
scores = cross_val_score(model,
                        diamonds[FEATS],
                        diamonds[TARGET],
                        scoring='neg_root_mean_squared_error',
                        cv=5, n_jobs=-1)

In [30]:
np.mean(-scores)

564.4805831354377

In [31]:
from sklearn.model_selection import RandomizedSearchCV 

In [32]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8, 16],
}

grid_search = RandomizedSearchCV(model, 
                                 param_grid, 
                                 cv=5, 
                                 verbose=10, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=32)

grid_search.fit(diamonds[FEATS], diamonds[TARGET])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   35.1s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  9.5min
/home/emmanuel/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to t

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer()),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'depth',
                                                                                'x',
                                                                                'y',
                                   

In [33]:
grid_search.best_params_

{'regressor__n_estimators': 512,
 'regressor__max_depth': 16,
 'preprocessor__num__imputer__strategy': 'mean'}

In [34]:
grid_search.best_score_

-559.5504664169242

In [35]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

In [36]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'],
                             'price': y_pred})

In [37]:
submission_df.head()

,id,price
0,0,2991.113078
1,1,5358.194714
2,2,9088.636558
3,3,4041.645497
4,4,1703.089067


In [38]:
submission_df.price.clip(370,19000, inplace=True)

In [39]:
submission_df.to_csv('./submissions/17-09.csv', index=False)